# Scraper

In [14]:
# Libraries
from bs4 import BeautifulSoup, Comment
import numpy as np
import pandas as pd
import urllib.request
import requests

## Helper Functions

In [15]:
# gets weekday of game give html div
def get_day(wk_day_span):
    wk_day_txt = wk_day_span.text.strip()
    date_components = wk_day_txt.split(' ')
    return(date_components[0])

In [16]:
# gets opposite of home line formatted correctly
def get_away_line(home_line):
    home_line = int(home_line) if float(home_line).is_integer() else float(home_line)
    if home_line < 0:
        return("{0:+}".format(home_line * -1))
    elif home_line > 0:
        return("{0:-}".format(home_line * -1))
    return('0')

## Test Code: Scraper

In [17]:
link = 'https://www.cbssports.com/nfl/scoreboard/all/2021/regular/' + str(4) + '/'
with urllib.request.urlopen(link) as url:
    page = url.read()
soup = BeautifulSoup(page, "html.parser")
# list of game info boxes for all games
score_boxes = soup.find_all('div', {'class':'live-update'})
score_box = score_boxes[0]
odds = score_box.find_all('td', {'class': 'in-progress-odds'})
over_under = odds[0].text.strip()[1:]
home_line = str(odds[1].text.strip())
away_line = float(home_line) * -1
teams = score_box.find_all('td', {'class': 'team'})
wk_day_span = score_box.find('span', {'class': 'game-status'})
#print(wk_day_span)
wk_day = get_day(wk_day_span)
#print(teams)
away_team = teams[0]
home_team = teams[1]
away_team_name = away_team.find_all('a')[1].text
away_team_abbr = away_team.find('a')['href'].split('/')[5]
home_team_name = home_team.find_all('a')[1].text
home_team_abbr = home_team.find('a')['href'].split('/')[5]
game_info_array = [home_team_name, home_team_abbr, away_team_name, away_team_abbr,
                 home_line, away_line, over_under, wk_day, str(4)]
print(game_info_array)

['Bengals', 'CIN', 'Jaguars', 'JAC', '-7.5', 7.5, '45.5', 'THU', '4']


## Scraping Functions

In [18]:
def get_game_info(score_box):
    odds = score_box.find_all('td', {'class': 'in-progress-odds'})
    over_under = odds[0].text.strip()[1:]
    home_line = odds[1].text.strip()
    away_line = get_away_line(home_line)
    teams = score_box.find_all('td', {'class': 'team'})
    wk_day_span = score_box.find('span', {'class': 'game-status'})
    wk_day = get_day(wk_day_span)
    away_team = teams[0]
    home_team = teams[1]
    away_team_name = away_team.find_all('a')[1].text
    away_team_abbr = away_team.find('a')['href'].split('/')[5]
    home_team_name = home_team.find_all('a')[1].text
    home_team_abbr = home_team.find('a')['href'].split('/')[5]
    game_info_array = [home_team_name, home_team_abbr, away_team_name, away_team_abbr,
                     home_line, away_line, over_under, wk_day]
    return(game_info_array)

In [19]:
def get_game_info_df(week_num):
    link = 'https://www.cbssports.com/nfl/scoreboard/all/2021/regular/' + str(week_num) + '/'
    with urllib.request.urlopen(link) as url:
        page = url.read()
    soup = BeautifulSoup(page, "html.parser")
    # list of game info boxes for all games
    score_boxes = soup.find_all('div', {'class':'live-update'})
    #game_info_list = np.array([])
    game_info_matrix = []
    for score_box in score_boxes:
        #print(score_box)
        game_info_matrix.append(get_game_info(score_box))
    cols = ['home_team_name', 'home_team_abbr', 'away_team_name', 
        'away_team_abbr', 'home_line', 'away_line', 
        'over_under', 'wk_day']
    game_info_df = pd.DataFrame(game_info_matrix, columns=cols)
    game_info_df['wk_num'] = str(week_num)
    return(game_info_df)

## Run Scraper

In [20]:
week_num = 4
df = get_game_info_df(week_num)
df

,home_team_name,home_team_abbr,away_team_name,away_team_abbr,home_line,away_line,over_under,wk_day,wk_num
0,Bengals,CIN,Jaguars,JAC,-7.5,+7.5,45.5,THU,4
1,Cowboys,DAL,Panthers,CAR,-4,+4,50,SUN,4
2,Vikings,MIN,Browns,CLE,+2,-2,53,SUN,4
3,Bears,CHI,Lions,DET,-3,+3,42.5,SUN,4
4,Bills,BUF,Texans,HOU,-16.5,+16.5,48,SUN,4
5,Dolphins,MIA,Colts,IND,-1.5,+1.5,43,SUN,4
6,Eagles,PHI,Chiefs,KC,+6.5,-6.5,55,SUN,4
7,Saints,NO,Giants,NYG,-7.5,+7.5,43.5,SUN,4
8,Jets,NYJ,Titans,TEN,+7.5,-7.5,46,SUN,4
9,Falcons,ATL,Football Team,WAS,+1.5,-1.5,48,SUN,4


# Export to Gsheets

In [21]:
import gspread

## Test Code: Export to Google Sheets

The following cell exports the df to a new worksheet within an existing google sheet. The creation of a new worksheet triggers the script editor within google sheets and create a corresponding google form.

In [117]:
# open connection to gsheets
gc = gspread.oauth(
        credentials_filename='.credentials/gspread/credentials.json',
        authorized_user_filename='.credentials/gspread/authorized_user.json',
    )
sh = gc.open("Pickem Lines Data")

# create new worksheet for the given week
worksheet_name = 'week' + str(week_num)
sh.add_worksheet(worksheet_name, rows=df.shape[0], cols=df.shape[1])
worksheet = sh.worksheet(worksheet_name)

# push df to google sheet
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1EjVIgaRpJQv8iDOxX4Sp4L32OpUXyng9NfUoKXRLF7M',
 'updatedRange': 'week4!A1:I17',
 'updatedRows': 17,
 'updatedColumns': 9,
 'updatedCells': 153}

## Export to Google Sheets Functions

In [65]:
def export_to_gsheets(df, week_num):
    # open connection to gsheets
    gc = gspread.oauth()
    sh = gc.open("Pickem Lines Data")

    # create new worksheet for the given week
    worksheet_name = 'week' + str(week_num)
    sh.add_worksheet(worksheet_name, rows=df.shape[0], cols=df.shape[1])
    worksheet = sh.worksheet(worksheet_name)

    # push df to google sheet
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

In [66]:
def run_export_process(week_num):
    df = get_game_info_df(week_num)
    export_to_gsheets(df, week_num)

## Test Code: Apps Script API

https://medium.com/@victor.perez.berruezo/execute-google-apps-script-functions-or-sheets-macros-programmatically-using-python-apps-script-ec8343e29fcd

In [131]:
import pickle
import os.path
from googleapiclient import errors
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [132]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/forms', 
          'https://www.googleapis.com/auth/spreadsheets']

In [141]:
def get_scripts_service():
    """Calls the Apps Script API.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('.credentials/scripts/token.pickle'):
        with open('.credentials/scripts/token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Credentials path from the credentials .json file 
            # from step 3 from Google Cloud Platform section
            flow = InstalledAppFlow.from_client_secrets_file(
                '.credentials/scripts/scripts_credentials.json', SCOPES) 
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('.credentials/scripts/token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('script', 'v1', credentials=creds)

In [142]:
service = get_scripts_service()

In [146]:
# API ID from step 3 in Google Sheets/Script section
from dotenv import load_dotenv
load_dotenv()

True

In [156]:
API_ID = os.getenv('API_ID')

In [157]:
# Instead macro_test select your macro function name 
# from step 5 in Sheets/Script section
request = {"function": "runProcess"} 

try:
    response = service.scripts().run(body=request, scriptId=API_ID).execute()
except errors.HttpError as error:
    # The API encountered a problem.
    print(error.content)

## Apps Script API Functions

In [158]:
def get_scripts_service():
    """Calls the Apps Script API.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('.credentials/scripts/token.pickle'):
        with open('.credentials/scripts/token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # Credentials path from the credentials .json file 
            # from step 3 from Google Cloud Platform section
            flow = InstalledAppFlow.from_client_secrets_file(
                '.credentials/scripts/scripts_credentials.json', SCOPES) 
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('.credentials/scripts/token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('script', 'v1', credentials=creds)

In [159]:
def run_apps_script():
    # If modifying these scopes, delete the file token.pickle.
    SCOPES = ['https://www.googleapis.com/auth/forms', 
              'https://www.googleapis.com/auth/spreadsheets']
    service = get_scripts_service()
    # PickEm project Scripts App API
    load_dotenv()
    API_ID = os.getenv('API_ID')
    request = {"function": "runProcess"}
    try:
        response = service.scripts().run(body=request, scriptId=API_ID).execute()
    except errors.HttpError as error:
        # The API encountered a problem.
        print(error.content)

## Export to GSheets, Create Form

In [160]:
week_num = 4
run_export_process(week_num)
run_apps_script()